In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Load CIFER-10 dataset

In [2]:
transform = transforms.Compose([
    transforms.Scale(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 Dataset
root = '/media/shuvendu/Projects/Datasets/CIFAR10'
train_dataset = dsets.CIFAR10(root=root,
                               train=True, 
                               transform=transform,
                               download=True)

test_dataset = dsets.CIFAR10(root=root,
                              train=False, 
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)



Using downloaded and verified file: /media/shuvendu/Projects/Datasets/CIFAR10/cifar-10-python.tar.gz


# Building the ResNet model

In [3]:
# Define a function for conv 3x3
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)


### Residual Block

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

### ResNet Module

In [5]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Training the model

In [6]:
resnet = ResNet(ResidualBlock, [3, 3, 3])
resnet.cuda()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)
    
# Training 
for epoch in range(80):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [%d/%d], Iter [%d/%d] Loss: %.4f" %(epoch+1, 80, i+1, 500, loss.data[0]))

    # Decaying Learning Rate
    if (epoch+1) % 20 == 0:
        lr /= 3
        optimizer = torch.optim.Adam(resnet.parameters(), lr=lr) 
        


Epoch [1/80], Iter [100/500] Loss: 1.6278
Epoch [1/80], Iter [200/500] Loss: 1.2787
Epoch [1/80], Iter [300/500] Loss: 1.1902
Epoch [1/80], Iter [400/500] Loss: 1.1375
Epoch [1/80], Iter [500/500] Loss: 1.0092
Epoch [2/80], Iter [100/500] Loss: 1.0454
Epoch [2/80], Iter [200/500] Loss: 0.8809
Epoch [2/80], Iter [300/500] Loss: 1.0690
Epoch [2/80], Iter [400/500] Loss: 1.1355
Epoch [2/80], Iter [500/500] Loss: 0.9469
Epoch [3/80], Iter [100/500] Loss: 0.9216
Epoch [3/80], Iter [200/500] Loss: 0.9826
Epoch [3/80], Iter [300/500] Loss: 0.8438
Epoch [3/80], Iter [400/500] Loss: 0.9510
Epoch [3/80], Iter [500/500] Loss: 0.8879
Epoch [4/80], Iter [100/500] Loss: 0.8309
Epoch [4/80], Iter [200/500] Loss: 0.8446
Epoch [4/80], Iter [300/500] Loss: 0.8098
Epoch [4/80], Iter [400/500] Loss: 0.7938
Epoch [4/80], Iter [500/500] Loss: 0.8441
Epoch [5/80], Iter [100/500] Loss: 0.8049
Epoch [5/80], Iter [200/500] Loss: 0.7304
Epoch [5/80], Iter [300/500] Loss: 0.8857
Epoch [5/80], Iter [400/500] Loss:

Epoch [39/80], Iter [300/500] Loss: 0.2652
Epoch [39/80], Iter [400/500] Loss: 0.2401
Epoch [39/80], Iter [500/500] Loss: 0.2367
Epoch [40/80], Iter [100/500] Loss: 0.0916
Epoch [40/80], Iter [200/500] Loss: 0.2862
Epoch [40/80], Iter [300/500] Loss: 0.1844
Epoch [40/80], Iter [400/500] Loss: 0.2550
Epoch [40/80], Iter [500/500] Loss: 0.2203
Epoch [41/80], Iter [100/500] Loss: 0.1757
Epoch [41/80], Iter [200/500] Loss: 0.1765
Epoch [41/80], Iter [300/500] Loss: 0.2603
Epoch [41/80], Iter [400/500] Loss: 0.1947
Epoch [41/80], Iter [500/500] Loss: 0.1836
Epoch [42/80], Iter [100/500] Loss: 0.2162
Epoch [42/80], Iter [200/500] Loss: 0.2081
Epoch [42/80], Iter [300/500] Loss: 0.1823
Epoch [42/80], Iter [400/500] Loss: 0.1525
Epoch [42/80], Iter [500/500] Loss: 0.1613
Epoch [43/80], Iter [100/500] Loss: 0.1790
Epoch [43/80], Iter [200/500] Loss: 0.1605
Epoch [43/80], Iter [300/500] Loss: 0.1309
Epoch [43/80], Iter [400/500] Loss: 0.1620
Epoch [43/80], Iter [500/500] Loss: 0.2098
Epoch [44/8

Epoch [77/80], Iter [400/500] Loss: 0.2009
Epoch [77/80], Iter [500/500] Loss: 0.2038
Epoch [78/80], Iter [100/500] Loss: 0.1712
Epoch [78/80], Iter [200/500] Loss: 0.1139
Epoch [78/80], Iter [300/500] Loss: 0.0981
Epoch [78/80], Iter [400/500] Loss: 0.1444
Epoch [78/80], Iter [500/500] Loss: 0.0762
Epoch [79/80], Iter [100/500] Loss: 0.1553
Epoch [79/80], Iter [200/500] Loss: 0.1402
Epoch [79/80], Iter [300/500] Loss: 0.1152
Epoch [79/80], Iter [400/500] Loss: 0.1715
Epoch [79/80], Iter [500/500] Loss: 0.1037
Epoch [80/80], Iter [100/500] Loss: 0.2773
Epoch [80/80], Iter [200/500] Loss: 0.1507
Epoch [80/80], Iter [300/500] Loss: 0.2299
Epoch [80/80], Iter [400/500] Loss: 0.1897
Epoch [80/80], Iter [500/500] Loss: 0.1976


# Test


In [7]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.cuda())
    outputs = resnet(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))

# Save the Model
torch.save(resnet.state_dict(), 'resnet.pkl')

Accuracy of the model on the test images: 86 %
